# Data load, clean and join

In [1]:
!pip install langid

In [2]:
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import os
import langid
import time

pd.set_option('display.max_columns', None)
pd.set_option('display.max_colwidth', 100)

In [3]:
games_df = pd.read_csv('games-original.csv', header=None, skiprows=1)
reviews_df = pd.read_csv('reviews-original.csv')

In [4]:
# Agrego columna "Unknown" ya que no viene nomenclada y genera un desfasaje en los indices de columnas
games_df_column_names = ['AppID', 'Name', 'Release date', 'Estimated owners', 'Peak CCU', 
                    'Required age', 'Price', 'Unknown', 'DiscountDLC count', 'About the game', 
                    'Supported languages', 'Full audio languages', 'Reviews', 'Header image', 
                    'Website', 'Support url', 'Support email', 'Windows', 'Mac', 
                    'Linux', 'Metacritic score', 'Metacritic url', 'User score', 
                    'Positive', 'Negative', 'Score rank', 'Achievements', 
                    'Recommendations', 'Notes', 'Average playtime forever', 
                    'Average playtime two weeks', 'Median playtime forever', 
                    'Median playtime two weeks', 'Developers', 'Publishers', 
                    'Categories', 'Genres', 'Tags', 'Screenshots', 'Movies']
games_df.columns = games_df_column_names

In [5]:
games_df.shape

(97410, 40)

In [6]:
reviews_df.shape

(6417106, 5)

In [7]:
games_df.dtypes

AppID                           int64
Name                           object
Release date                   object
Estimated owners               object
Peak CCU                        int64
Required age                    int64
Price                         float64
Unknown                         int64
DiscountDLC count               int64
About the game                 object
Supported languages            object
Full audio languages           object
Reviews                        object
Header image                   object
Website                        object
Support url                    object
Support email                  object
Windows                          bool
Mac                              bool
Linux                            bool
Metacritic score                int64
Metacritic url                 object
User score                      int64
Positive                        int64
Negative                        int64
Score rank                    float64
Achievements

In [8]:
reviews_df.dtypes

app_id           int64
app_name        object
review_text     object
review_score     int64
review_votes     int64
dtype: object

In [9]:
games_df.head()

,AppID,Name,Release date,Estimated owners,Peak CCU,Required age,Price,Unknown,DiscountDLC count,About the game,Supported languages,Full audio languages,Reviews,Header image,Website,Support url,Support email,Windows,Mac,Linux,Metacritic score,Metacritic url,User score,Positive,Negative,Score rank,Achievements,Recommendations,Notes,Average playtime forever,Average playtime two weeks,Median playtime forever,Median playtime two weeks,Developers,Publishers,Categories,Genres,Tags,Screenshots,Movies
0,20200,Galactic Bowling,"Oct 21, 2008",0 - 20000,0,0,19.99,0,0,Galactic Bowling is an exaggerated and stylized bowling game with an intergalactic twist. Player...,['English'],[],NaN,https://cdn.akamai.steamstatic.com/steam/apps/20200/header.jpg?t=1640121033,http://www.galacticbowling.net,NaN,NaN,True,False,False,0,NaN,0,6,11,NaN,30,0,NaN,0,0,0,0,Perpetual FX Creative,Perpetual FX Creative,"Single-player,Multi-player,Steam Achievements,Partial Controller Support","Casual,Indie,Sports","Indie,Casual,Sports,Bowling","https://cdn.akamai.steamstatic.com/steam/apps/20200/0000005994.1920x1080.jpg?t=1640121033,https:...",http://cdn.akamai.steamstatic.com/steam/apps/256863704/movie_max.mp4?t=1638854607
1,655370,Train Bandit,"Oct 12, 2017",0 - 20000,0,0,0.99,0,0,"THE LAW!! Looks to be a showdown atop a train. This will be your last fight. Good luck, Train Ba...","['English', 'French', 'Italian', 'German', 'Spanish - Spain', 'Japanese', 'Portuguese - Brazil',...",[],NaN,https://cdn.akamai.steamstatic.com/steam/apps/655370/header.jpg?t=1617500526,http://trainbandit.com,NaN,support@rustymoyher.com,True,True,False,0,NaN,0,53,5,NaN,12,0,NaN,0,0,0,0,Rusty Moyher,Wild Rooster,"Single-player,Steam Achievements,Full controller support,Steam Leaderboards,Remote Play on Phone...","Action,Indie","Indie,Action,Pixel Graphics,2D,Retro,Arcade,Score Attack,Minimalist,Comedy,Singleplayer,Fast-Pac...",https://cdn.akamai.steamstatic.com/steam/apps/655370/ss_16785eac54b29db688870e832f6997c89cd7804b...,http://cdn.akamai.steamstatic.com/steam/apps/256691108/movie_max.mp4?t=1506089586
2,1732930,Jolt Project,"Nov 17, 2021",0 - 20000,0,0,4.99,0,0,"Jolt Project: The army now has a new robotics project, jolt. It's up to you to control it and en...","['English', 'Portuguese - Brazil']",[],NaN,https://cdn.akamai.steamstatic.com/steam/apps/1732930/header.jpg?t=1637149386,NaN,NaN,ramoncampiaof31@gmail.com,True,False,False,0,NaN,0,0,0,NaN,0,0,NaN,0,0,0,0,Campião Games,Campião Games,Single-player,"Action,Adventure,Indie,Strategy",NaN,https://cdn.akamai.steamstatic.com/steam/apps/1732930/ss_09d67dec0607be7c4ead80289763033a47c86d8...,"http://cdn.akamai.steamstatic.com/steam/apps/256847488/movie_max.mp4?t=1635980739,http://cdn.aka..."
3,1355720,Henosis™,"Jul 23, 2020",0 - 20000,0,0,5.99,0,0,HENOSIS™ is a mysterious 2D Platform Puzzler where players are propelled into weird and visceral...,"['English', 'French', 'Italian', 'German', 'Spanish - Spain', 'Japanese', 'Korean', 'Portuguese'...",[],NaN,https://cdn.akamai.steamstatic.com/steam/apps/1355720/header.jpg?t=1639875115,https://henosisgame.com/,https://henosisgame.com/,info@henosisgame.com,True,True,True,0,NaN,0,3,0,NaN,0,0,NaN,0,0,0,0,Odd Critter Games,Odd Critter Games,"Single-player,Full controller support","Adventure,Casual,Indie","2D Platformer,Atmospheric,Surreal,Mystery,Puzzle,Survival,Adventure,Linear,Singleplayer,Experime...",https://cdn.akamai.steamstatic.com/steam/apps/1355720/ss_20bbae2d9d5aaa2f043f372a551faabc0c47b0b...,http://cdn.akamai.steamstatic.com/steam/apps/256819153/movie_max.mp4?t=1611314333
4,1139950,Two Weeks in Painland,"Feb 3, 2020",0 - 20000,0,0,0.00,0,0,ABOUT THE GAME Play as a hacker who has arranged a deal with a gangster. That’s how the protagon...,"['English', 'Spanish - Spain']",[],NaN,https://cdn.akamai.steamstatic.com/steam/apps/1139950/header.jpg?t=1595003825,https://www.unusual-games.com/home/,https://www.unusual-games.com/contact/,welistentoyou@unusual-games.com,True,True,False,0,NaN,0,50,8,Na

In [10]:
reviews_df.head()

,app_id,app_name,review_text,review_score,review_votes
0,10,Counter-Strike,Ruined my life.,1,0
1,10,Counter-Strike,"This will be more of a ''my experience with this game'' type of review, because saying things li...",1,1
2,10,Counter-Strike,This game saved my virginity.,1,0
3,10,Counter-Strike,• Do you like original games? • Do you like games that don't lag? • Do you like games you can ru...,1,0
4,10,Counter-Strike,"Easy to learn, hard to master.",1,1


In [11]:
games_df['AppID'].nunique()

97410

In [12]:
reviews_df["app_id"].nunique()

9972

In [13]:
reviews_df["review_score"].value_counts()

review_score
 1    5260420
-1    1156686
Name: count, dtype: int64

In [14]:
# Dataframes cleaning

In [15]:
games_df_columns = ['AppID', 'Name', 'Windows', 'Mac', 'Linux', 'Genres', 'Release date', 'Average playtime forever', 'Positive', 'Negative']
reviews_df_columns = ['app_id', 'review_text', 'review_score']

In [16]:
games_df_cleaned = games_df.dropna(subset=games_df_columns)[games_df_columns].copy()
reviews_df_cleaned = reviews_df.dropna(subset=reviews_df_columns)[reviews_df_columns].copy()

In [17]:
games_df_cleaned["Genres"] = games_df_cleaned["Genres"].str.lower()

In [18]:
reviews_df_cleaned['review_text'] = reviews_df_cleaned['review_text'].astype(str)

# Queries resolution

## Q1: Cantidad de juegos soportados en cada plataforma (Windows, Linux, MAC)

In [19]:
windows_supported_games = games_df_cleaned[games_df_cleaned["Windows"] == True]
linux_supported_games = games_df_cleaned[games_df_cleaned["Linux"] == True]
mac_supported_games = games_df_cleaned[games_df_cleaned["Mac"] == True]

In [20]:
print("Total de juegos: " + str(games_df_cleaned.shape[0]))
print("Total de juegos soportados en Windows: " + str(windows_supported_games.shape[0]))
print("Total de juegos soportados en Linux: " + str(linux_supported_games.shape[0]))
print("Total de juegos soportados en Mac: " + str(mac_supported_games.shape[0]))

Total de juegos: 92565
Total de juegos soportados en Windows: 92534
Total de juegos soportados en Linux: 12307
Total de juegos soportados en Mac: 17703


## Q2: Nombre del top 10 de juegos del género "Indie" publicados en la década del 2010 con más tiempo promedio histórico de juego

In [21]:
games_indie = games_df_cleaned[games_df_cleaned["Genres"].str.contains("indie")]

In [22]:
games_indie_2010_decade = games_indie[games_indie["Release date"].str.contains("201")]

In [23]:
games_indie.shape

(64498, 10)

In [24]:
games_indie_2010_decade.shape

(23195, 10)

In [25]:
q2_result = games_indie_2010_decade.sort_values(by='Average playtime forever', ascending=False).head(10)

In [26]:
q2_result[['Name', 'Average playtime forever']]

,Name,Average playtime forever
39591,Relive,68357
7529,拯救大魔王2 Rescue the Great Demon 2,49555
18525,Out of the Park Baseball 20,34798
21216,时之回廊/Corridor of time,33343
4299,Clicker Heroes 2,31708
78237,Time of Dragons,27080
53439,Project AURA,26827
2187,Midas Gold Plus,19196
4771,Beasts of Bermuda,17147
35001,Rust,16623


## Q3: Nombre de top 5 juegos del género "Indie" con más reseñas positivas

In [27]:
games_indie_reduced = games_indie[["AppID", "Name"]]
games_indie_reduced.head()

,AppID,Name
0,20200,Galactic Bowling
1,655370,Train Bandit
2,1732930,Jolt Project
3,1355720,Henosis™
4,1139950,Two Weeks in Painland


In [28]:
reviews_reduced_q3 = reviews_df_cleaned[["app_id", "review_score"]]

In [29]:
games_indie_reviews = pd.merge(games_indie_reduced, reviews_reduced_q3, left_on='AppID', right_on='app_id', how='inner')

In [30]:
def positive_score(score):
    return 1 if score > 0 else 0

games_indie_reviews['positive_score'] = games_indie_reviews['review_score'].apply(positive_score)

In [31]:
q3_result = games_indie_reviews.groupby('Name')['positive_score'].sum().sort_values(ascending=False).head(5)

In [32]:
q3_result.head(10)

Name
Terraria          82227
Rust              61032
Rocket League®    51084
Undertale         49812
Starbound         36134
Name: positive_score, dtype: int64

## Q4: Nombre de juegos del género "action" con más de 5.000 reseñas negativas en idioma inglés

In [48]:
# Juegos de acción
games_action = games_df_cleaned[games_df_cleaned["Genres"].str.contains("action")]
games_action_reduced = games_action[["AppID", "Name"]]
games_action_reduced.shape

(38002, 2)

In [49]:
reviews_q4 = reviews_df_cleaned.copy()

In [50]:
# Reviews con mas de 5000 comentarios negativos

def negative_score(score):
    return 1 if score < 0 else 0

reviews_q4["negative_score"] = reviews_q4["review_score"].apply(negative_score)
reviews_q4_negatives = reviews_q4[reviews_q4["negative_score"] == 1].copy()
reviews_count = reviews_q4_negatives.groupby('app_id').size().reset_index(name='count')
reviews_count_more_than_5000 = reviews_count[reviews_count["count"] > 5000]
reviews_count_more_than_5000.shape

(23, 2)

In [51]:
# De las reviews con mas de 5000 comentarios negativos, nos quedamos con aquellas que sean sobre juegos de acción
games_action_with_5000_negative_reviews = pd.merge(games_action_reduced, reviews_count_more_than_5000, left_on='AppID', right_on="app_id", how='inner')
games_action_with_5000_negative_reviews = games_action_with_5000_negative_reviews[["AppID", "Name"]]
games_action_with_5000_negative_reviews.shape

(20, 2)

In [52]:
# Enriquecemos con el texto de la review
reviews_count_more_than_5000_with_text = pd.merge(reviews_q4, games_action_with_5000_negative_reviews, left_on='app_id', right_on="AppID", how='inner')
reviews_count_more_than_5000_with_text = reviews_count_more_than_5000_with_text[["app_id", "review_text"]]
reviews_count_more_than_5000_with_text.shape

(672593, 2)

In [54]:
# CPU INTENSIVE #############################
def detect_language(texto):
    language, _ = langid.classify(texto)
    return language
#############################################

# Calculo del idioma sobre las reviews
start_time = time.time()
reviews_count_more_than_5000_with_text["review_language"] = reviews_count_more_than_5000_with_text['review_text'].apply(detect_language)
elapsed_time = time.time() - start_time
print(f"Execution time on {reviews_count_more_than_5000_with_text.shape[0]} rows: {elapsed_time:.2f} seconds")

# Con el dataset original y más de 5000 reviews tarda 722.20 segundos
# Con el dataset reducido y más de 250 reviews tarda x segundos

Execution time on 672593 rows: 756.79 seconds


In [55]:
reviews_count_more_than_5000_with_text.shape

(672593, 3)

In [56]:
# Nos quedamos con aquellas reviews que estan en idioma inglés
reviews_count_more_than_5000_with_text_english = reviews_count_more_than_5000_with_text[reviews_count_more_than_5000_with_text["review_language"] == "en"]
reviews_count_more_than_5000_with_text_english.shape

(658057, 3)

In [57]:
# Nos quedamos con aquellos juegos que tengan mas de 5000 reseñas negativas en inglés
q4_results_app_ids = reviews_count_more_than_5000_with_text_english.groupby('app_id').size().reset_index(name='count')
q4_results_app_ids = q4_results_app_ids[q4_results_app_ids["count"] > 5000]
q4_results_app_ids.head(25)

,app_id,count
0,570,68946
1,12210,13766
2,113400,12000
3,208650,16660
4,211820,40996
5,218620,85382
6,221100,88850
7,227940,31178
8,247730,9355
9,252490,77037


In [58]:
# Enriquecemos con el nombre de esos juegos
q4_results_games_names = pd.merge(q4_results_app_ids, games_action_with_5000_negative_reviews, left_on='app_id', right_on="AppID", how='inner')["Name"]
q4_results_games_names.head(25)

0                                        Dota 2
1     Grand Theft Auto IV: The Complete Edition
2                                  APB Reloaded
3                        Batman™: Arkham Knight
4                                     Starbound
5                                      PAYDAY 2
6                                          DayZ
7                             Heroes & Generals
8                           Nether: Resurrected
9                                          Rust
10                            theHunter Classic
11                           Grand Theft Auto V
12                               Evolve Stage 2
13                                 No Man's Sky
14                                    Robocraft
15                 Call of Duty®: Black Ops III
16                        ARK: Survival Evolved
17                   Tom Clancy’s The Division™
18                             Dead by Daylight
19                             Z1 Battle Royale
Name: Name, dtype: object

## Q5: Nombre de juegos del género "action" dentro del percentil 90 en cantidad de reseñas negativas

In [38]:
games_action_reduced.head()

,AppID,Name
1,655370,Train Bandit
2,1732930,Jolt Project
9,320150,Deadlings: Rotten Edition
14,1454010,Diary of Lucie
15,22670,Alien Breed 3: Descent


In [39]:
games_action_reduced.shape

(38002, 2)

In [40]:
reviews_q5 = reviews_df_cleaned.copy()
reviews_q5 = reviews_q5[["app_id", "review_score"]]
reviews_q5["negative_score"] = reviews_q5["review_score"].apply(negative_score)
reviews_q5_negative_score = reviews_q5[reviews_q5["negative_score"] == 1]
reviews_q5_negative_score.shape

(1155685, 3)

In [41]:
reviews_q5_negative_score_action = pd.merge(reviews_q5_negative_score, games_action_reduced, left_on='app_id', right_on="AppID", how='inner')
reviews_q5_negative_score_action.shape

(717190, 5)

In [42]:
reviews_q5_negative_score_action_by_app_id = reviews_q5_negative_score_action.groupby('app_id').size().reset_index(name='count')
reviews_q5_negative_score_action_by_app_id.shape

(3912, 2)

In [43]:
percentil_90 = reviews_q5_negative_score_action_by_app_id['count'].quantile(0.90)
percentil_90

np.float64(306.60000000000036)

In [44]:
q5_result = reviews_q5_negative_score_action_by_app_id[reviews_q5_negative_score_action_by_app_id['count'] >= percentil_90]
q5_result.shape

(392, 2)

In [45]:
q5_result_with_game_names = pd.merge(q5_result, games_action_reduced, left_on='app_id', right_on="AppID", how='inner')
q5_result_with_game_names[["app_id", "Name"]].head(10)

,app_id,Name
0,10,Counter-Strike
1,240,Counter-Strike: Source
2,280,Half-Life: Source
3,340,Half-Life 2: Lost Coast
4,440,Team Fortress 2
5,550,Left 4 Dead 2
6,570,Dota 2
7,620,Portal 2
8,1250,Killing Floor
9,2400,The Ship: Murder Party
